# setup

In [170]:
#!g1.1
# %pip install transformers==4.27.1

In [ ]:
#!g1.1
!nvidia-smi

In [215]:
#!g1.1
!rm -rf .cache/

In [216]:
#!g1.1
%run -n main.py
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline
)

model_name = GUSEV_LARGE_TURBO
cache_dir = '.cache'
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir=cache_dir)
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0)

In [225]:
#!g1.1
tokenizer.pad_token = '<pad>'
tokenizer.padding_side = 'left'

In [226]:
#!g1.1
from transformers import GenerationConfig

generation_config = GenerationConfig(
    max_new_tokens=32,
    do_sample=True,
    
    top_p=0.95,
    temperature=0.5,
    repetition_penalty=1.2,

    pad_token_id=tokenizer.pad_token_id
)

In [227]:
#!g1.1
prompt = '''
Однажды
'''
output = generator(
    prompt,
    generation_config=generation_config,
    return_full_text=False,
    num_return_sequences=5
)
output

[{'generated_text': 'Когда-то я был таким же, как ты. Я любил свою работу и мечтал о том дне, когда смогу уйти на пенсию. Но однажды я встретил своего'},
 {'generated_text': 'Имя: Андрей Имя: Андрей, привет! Как дела? Я тут недавно узнал о твоем проекте. Расскажи мне все подробности и я сделаю тебе предложение!'},
 {'generated_text': 'у меня была идея, что я могу сделать для своего города. Я начал с того, чтобы найти людей, которые хотели бы работать в моем городе и помочь им'},
 {'generated_text': 'в парке, где много деревьев и цветов, я увидел прекрасную картину. На скамейке сидела девушка с книгой в руках. Вокруг нее были разбросаны цветы и листья.'},
 {'generated_text': 'я шел по улице и увидел женщину, которая сидела на скамейке. Я подошел к ней и спросил: "Что случилось?" Она ответила мне: "Я потеряла своего'}]

# eval

In [220]:
#!g1.1
%run -n main.py
task = TERRA
count = len(list(Path('evals').glob('*.jsonl')))
eval = f'{count + 1}_large_turbo_{task}'
eval

'84_large_turbo_terra'

In [221]:
#!g1.1
%run -n main.py
lines = read_lines(f'tasks/{task}.jsonl')
task_items = list(parse_jsonl(lines))

In [228]:
#!g1.1
%run -n main.py
prompts = [
    TASK_PROMPTS[task](_)
    for _ in task_items
]
prompt = random.choice(prompts)
print(prompt)

Прочитай текст, проверь верно ли утверждение.
Ответь коротко: да или нет. Если не уверен, выбери наиболее вероятный ответ.
---
Текст: Трижды он был привлечён судебным приставом к административной ответственности по ст. 17.15 КоАП РФ за неисполнение содержащихся в исполнительном документе требований неимущественного характера. Так как срок для добровольного исполнения истёк, пристрой снесли принудительно.
Утверждение: Пристрой был снесен.
Верно: Да
---
Текст: Для молодого организма это не прошло бесследно. Резкое токсическое воздействие этанола привело к смерти парня. Его тело обнаружила бабушка, которая вернулась на следующий день.
Утверждение: Молодой организм стал сильнее от этанола.
Верно: Нет
---
Текст: В зарубежной практике примеров подобной нормы, по его мнению, не встречается. В мировой практике акционерное соглашение дополняет или замещает некоторые элементы учредительных документов. Делается это в том числе для того, чтобы сохранить некоторые положения соглашений конфиденциаль

In [229]:
#!g1.1
generator(
    prompt,
    generation_config=generation_config,
    return_full_text=False,
    num_return_sequences=4
)

[{'generated_text': '^\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'},
 {'generated_text': '\nДа\x00\x00\x03\x02�‘s\x1d¿\x07� │\n[id285'},
 {'generated_text': '\nНет\x00\x1d\x00\x03�\x00\x0b�\x00!\x00\x03�\x00!\x03�\x00!\x16�'},
 {'generated_text': '\nНет\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'}]

In [230]:
#!g1.1
output = generator(
    prompts,
    generation_config=generation_config,
    return_full_text=False,
    batch_size=8
)

In [231]:
#!g1.1
random.sample(output, 3)

[[{'generated_text': '\nНет\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'}],
 [{'generated_text': '\nНет\x00\x00\x03\x00\x03\x02��\x00\x0b��\x00\x0f\x00\x1d\x00̃�'}],
 [{'generated_text': '````````````````````````````````'}]]

In [232]:
#!g1.1
eval_items = []
for task_item, response_item in zip(task_items, output):
    response = response_item[0]['generated_text']

    eval_items.append({
        'id': task_item['id'],
        'response': response
    })

In [235]:
#!g1.1
%run -n main.py
id_targets = {
    _['id']: _['label']
    for _ in task_items
}
norm_response = NORM_RESPONSES[task]
id_preds = {
    _['id']: norm_response(_['response'])
    for _ in eval_items
}
acc_score(id_targets, id_preds)

(0.43548387096774194, 38)

In [236]:
#!g1.1
lines = format_jsonl(eval_items)
write_lines(f'evals/{eval}.jsonl', lines)

In [237]:
#!g1.1
for task in TASKS[1:]:
    count = len(list(Path('evals').glob('*.jsonl')))
    eval = f'{count + 1}_large_turbo_{task}'

    lines = read_lines(f'tasks/{task}.jsonl')
    task_items = list(parse_jsonl(lines))
    
    prompts = [
        TASK_PROMPTS[task](_)
        for _ in task_items
    ]
    
    output = generator(
        prompts,
        generation_config=generation_config,
        return_full_text=False,
        batch_size=8
    )
    
    eval_items = []
    for task_item, response_item in zip(task_items, output):
        response = response_item[0]['generated_text']
        eval_items.append({
            'id': task_item['id'],
            'response': response
        })
    
    lines = format_jsonl(eval_items)
    write_lines(f'evals/{eval}.jsonl', lines)

/home/jupyter/.local/lib/python3.8/site-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
#!g1.1
